# MFCC Feature Extraction

Extract mfcc coeffecients from both normal and augmented segments

In [7]:
import os
import numpy as np
import librosa
import soundfile as sf
import pandas as pd
from pathlib import Path
from tqdm import tqdm

In [8]:
def extract_mfcc_features(segment_info_path, segments_dir='../../segments', n_mfcc=13, features_output_dir='../../extracted_features/features/mfcc_features.npy', labels_output_dir='../../extracted_features/labels/mfcc_labels.npy'):
    """
    Extract MFCC features from segmented audio files.
    
    Args:
        segment_info_path: Path to segment_info.csv
        segments_dir: Directory containing the audio segments (either 'segments' or 'augmentedSegments')
        n_mfcc: Number of MFCC coefficients to compute
    """
    # Load segment info
    metadata = pd.read_csv(segment_info_path)
    
    # Update paths to use the specified segments directory
    segments_path = Path(segments_dir)
    metadata['segment_path'] = metadata['segment_path'].apply(
        lambda x: str(segments_path / Path(x).name))
    
    # Initialize arrays to store features and labels
    features = []
    labels = []
    
    print(f"Extracting MFCC features from {segments_dir}...")
    for idx, row in tqdm(metadata.iterrows(), total=len(metadata)):
        try:
            # Load audio segment
            y, sr = librosa.load(row['segment_path'])
            
            # Extract MFCCs
            mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
            
            # Take mean of each coefficient over time
            mfcc_mean = np.mean(mfcc, axis=1)
            
            features.append(mfcc_mean)
            labels.append(row['instrument_label'])
            
        except Exception as e:
            print(f"Error processing {row['segment_path']}: {str(e)}")
            metadata = metadata.drop(idx)
            continue
    
    # Convert to numpy arrays
    X = np.array(features)
    y = np.array(labels)
    
    # Create features directory if it doesn't exist
    # output_dir = Path('features')
    # output_dir.mkdir(exist_ok=True, parents=True)
    
    # Save features and labels with directory-specific names
    np.save(features_output_dir, X)
    np.save(labels_output_dir, y)
    
    return X, y, metadata

In [11]:
segment_info_path = '../../segment_info/segment_info.csv'
augmented_segment_info_path = '../../segment_info/augmented_segment_info.csv'

os.makedirs('../../extracted_features/features', exist_ok=True)
os.makedirs('../../extracted_features/labels', exist_ok=True)

X_mfcc, y_mfcc, metadata_mfcc = extract_mfcc_features(
    segment_info_path,
    segments_dir='../../segments',
    features_output_dir='../../extracted_features/features/mfcc_features.npy',
    labels_output_dir='../../extracted_features/labels/mfcc_labels.npy'
)

Extracting MFCC features from ../../segments...


100%|██████████| 5714/5714 [00:08<00:00, 637.75it/s]


In [10]:
X_mfcc_aug, y_mfcc_aug, metadata_mfcc_aug = extract_mfcc_features(
    augmented_segment_info_path,
    segments_dir='../../augmentedSegments',
    features_output_dir='../../extracted_features/features/mfcc_features_aug.npy',
    labels_output_dir='../../extracted_features/labels/mfcc_labels_aug.npy'
)

Extracting MFCC features from ../../augmentedSegments...


100%|██████████| 34284/34284 [00:49<00:00, 689.28it/s]
